In [25]:
!curl -L https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data -o letter-recognition.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  695k  100  695k    0     0   224k      0  0:00:03  0:00:03 --:--:--  224k


In [26]:
# importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import scale
from math import exp
from collections import defaultdict 
from sklearn.metrics.pairwise import chi2_kernel
# import warnings
# warnings.filterwarnings("ignore")

In [27]:
df = pd.read_csv('letter-recognition.csv')

# feature names were like - 'letter ' changed it to 'letter'
df.columns = ['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar',
       'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge',
       'xedgey', 'yedge', 'yedgex']
print(df.columns)

Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')


In [28]:
print(f"Number of rows = {df.shape[0]}, cols = {df.shape[1]}")
df.head()

Number of rows = 19999, cols = 17


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
1,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
2,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
3,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
4,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7


In [29]:
# number of different classes
classes = df['letter'].unique()
print(f'Number of different labels in dataset = {len(classes)}')
classes.sort()
for cls in classes:
    rows = len(df[df['letter'] == cls])
    print(f'Number of data points in class {cls} = {rows}')

Number of different labels in dataset = 26
Number of data points in class A = 789
Number of data points in class B = 766
Number of data points in class C = 736
Number of data points in class D = 805
Number of data points in class E = 768
Number of data points in class F = 775
Number of data points in class G = 773
Number of data points in class H = 734
Number of data points in class I = 755
Number of data points in class J = 747
Number of data points in class K = 739
Number of data points in class L = 761
Number of data points in class M = 792
Number of data points in class N = 783
Number of data points in class O = 753
Number of data points in class P = 803
Number of data points in class Q = 783
Number of data points in class R = 758
Number of data points in class S = 748
Number of data points in class T = 795
Number of data points in class U = 813
Number of data points in class V = 764
Number of data points in class W = 752
Number of data points in class X = 787
Number of data points

In [30]:
def predict(z):
  x = zt.loc[:, zt.columns != 'letter']
  y = zt.loc[:, zt.columns == 'letter']
  # doubtful
  fx = f0.decision_function(x)/w_norm
  return fx[0]

In [31]:
# returns the loss value calculated using model f0
def loss(z):
  y = z.iloc[0]['letter']
  # doubtful
  fx = predict(z)

  if fx*y > 1:
    return 0
  else:
    return 1-fx*y

In [32]:
def calculate_P(zs,zst):
  P = exp(-loss(zs))/exp(-loss(zt))
  return P

In [33]:
def acceptance_prob(zs,zt):
  ys = zs.iloc[0]['letter']
  yt = zt.iloc[0]['letter']

  fs = predict(zs)
  ft = predict(zt)

  return exp(-fs*ys)/exp(-ft*yt)

In [34]:
def hellinger(X1, X2):
  return np.sqrt(np.dot(X1,X2.T))

In [35]:
avg_acc = {'linear':0, 'rbf':0, 'poly':0, 'chi_squared':0, 'hellinger':0}

for alp in classes:
    print(f'\nRunning for alphabet = {alp}')

    df1 = df.copy()

    alphabet = alp
    df1.loc[(df1.letter != alphabet),'letter'] = -1
    df1.loc[(df1.letter == alphabet),'letter'] = 1
    pos = df1[df1['letter']==1].sample(frac=1)
    neg = df1[df1['letter']==-1].sample(frac=1/25)
    df_final = pos.append(neg,ignore_index=True)

    df_final_train, df_final_test = train_test_split(df_final, test_size=0.3, random_state=42)
    M = 1000
    N1 = round(M * 0.75)
    df_sample = df_final_train.sample(n=N1)
    df_sample_features = df_sample.loc[:, df_sample.columns != 'letter']
    df_sample_target = df_sample.loc[:, ['letter']]

    # df_sample_features = scale(df_sample_features)
    df_sample_target = df_sample_target.astype('int')

    f0 = SVC(kernel = 'linear')
    f0.fit(df_sample_features, df_sample_target.values.ravel())
    w_norm = np.linalg.norm(f0.coef_) 

    m_pos, m_neg = 0, 0
    zt = df_final_train.sample(n=1)
    # print(zt)
    if M%2 == 0:
        label = zt.iloc[0]['letter']
        print("")
        if label == 1:
            m_pos += 1
        else:
            m_neg += 1

    # Step 3
    # should its type be pandas.dataframe?
    sampled_data = []
    # keep count of how many times a sample is rejected
    rejected = defaultdict(int)

    # all variable names are as given in paper
    # constants used in algorithm, see section 4A point 1 in paper
    K = 5
    Q = 1.2
    while m_pos < M//2 or m_neg < M//2:
        # draw one sample randomly = z_star(zs)
        zs = df_final_train.sample(n=1)
        # print("considering ",zs)
        P = calculate_P(zs,zt)
        
        # yt corresponds to Y of zt
        yt = zt.iloc[0]['letter']
        ys = zs.iloc[0]['letter']

        # alpha is the acceptance probability of zs, given in step 5
        alpha = P
        # need to convert it to tuple so that it becomes hashable
        tpl = tuple(zs.to_records(index=False)[0])

        if P==1 and yt==-1 and ys==-1:
            alpha = acceptance_prob(zs,zt)  # P'
        elif P==1 and yt==1 and ys==1:    
            alpha = acceptance_prob(zs,zt)  # P'
        elif (P==1 and yt*ys==-1) or P<1 :
            alpha = P                       # P
        elif rejected[tpl]>K:
            alpha = Q*P                      # P''
                                
        if alpha>1:
            alpha=1

        # print(f"alpha {alpha}")
        if np.random.random() < alpha:
            sampled_data.append(zs)
        else:
            rejected[tpl]+=1

        if yt == 1:
            m_pos += 1 
        else:
            m_neg += 1
        
        # for next iteration 
        zt = zs
        # print(f"Positive = {m_pos} Negative = {m_neg}")

    dataset = pd.concat(sampled_data)
    features = dataset.loc[:, dataset.columns != 'letter']
    target = dataset.loc[:, ['letter']]

    # X_train = scale(features)
    X_train = features
    y_train = target
    y_train = y_train.astype('int')


    # X_test = scale(df_final_test.loc[:, df_final_test.columns != 'letter'])
    X_test = df_final_test.loc[:, df_final_test.columns != 'letter']
    y_test = df_final_test.loc[:, ['letter']]
    y_test = y_test.astype('int')


    # model training
    import warnings
    warnings.filterwarnings("ignore")
    kernels = ['linear', 'rbf', 'poly']
    for kernel in kernels:
        model = SVC(kernel = kernel)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_true = y_test, y_pred = y_pred)
        print(f"Kernel = {kernel}, accuracy = {acc * 100}")
        avg_acc[kernel] += acc

    model = SVC(kernel = hellinger)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_true = y_test, y_pred = y_pred)
    print(f"Kernel = hellinger, accuracy = {acc * 100}")
    avg_acc['hellinger'] += acc

    model = SVC(kernel = chi2_kernel)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_true = y_test, y_pred = y_pred)
    print(f"Kernel = chi2_kernel, accuracy = {acc * 100}")
    avg_acc['chi_squared'] += acc

print()
for item in avg_acc:
    val = avg_acc[item] / 26
    print(f"Average accuracy for {item} = {val * 100}")


Running for alphabet = A

Kernel = linear, accuracy = 95.08547008547008
Kernel = rbf, accuracy = 95.51282051282051
Kernel = poly, accuracy = 95.72649572649573
Kernel = hellinger, accuracy = 93.58974358974359
Kernel = chi2_kernel, accuracy = 95.08547008547008

Running for alphabet = B

Kernel = linear, accuracy = 90.67245119305856
Kernel = rbf, accuracy = 93.70932754880694
Kernel = poly, accuracy = 95.66160520607376
Kernel = hellinger, accuracy = 88.28633405639913
Kernel = chi2_kernel, accuracy = 96.31236442516268

Running for alphabet = C

Kernel = linear, accuracy = 90.50772626931567
Kernel = rbf, accuracy = 94.03973509933775
Kernel = poly, accuracy = 92.27373068432672
Kernel = hellinger, accuracy = 90.06622516556291
Kernel = chi2_kernel, accuracy = 96.68874172185431

Running for alphabet = D

Kernel = linear, accuracy = 90.67796610169492
Kernel = rbf, accuracy = 95.12711864406779
Kernel = poly, accuracy = 95.12711864406779
Kernel = hellinger, accuracy = 87.71186440677965
Kernel = ch